In [1]:
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.model_selection import KFold
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import auc

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from IPython.display import clear_output

In [2]:
# Entscheidungsbaum
TREE_PARAMS = {
    "random_state" : 0,
    "max_depth": 7
}

In [3]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [4]:
app_train = pd.read_csv(DATASET_DIR / "Datenaufbereitung" / "app_train_mets.csv", index_col="SK_ID_CURR")
bureau = pd.read_csv(DATASET_DIR / "Datenaufbereitung" / "bureau_mets.csv", index_col="SK_ID_CURR")
pa = pd.read_csv(DATASET_DIR / "Datenaufbereitung" / "pa_mets.csv", index_col="SK_ID_CURR")
ip = pd.read_csv(DATASET_DIR / "Datenaufbereitung" / "ip_mets.csv", index_col="SK_ID_CURR")
pos = pd.read_csv(DATASET_DIR / "Datenaufbereitung" / "pos_mets.csv", index_col="SK_ID_CURR")

In [5]:
ip = ip.drop("TARGET", axis=1)
pos = pos.drop("TARGET", axis=1)

In [6]:
pos.head()

,CNT_CREDITS_PAST,CNT_CREDITS_CURR,SUM_PAYCOUNTS_MONTHS,SUM_DPD
SK_ID_CURR,,,,
182943,1.0,1.0,15.0,0
367990,2.0,0.0,0.0,0
397406,7.0,1.0,1.0,4110
269225,5.0,1.0,39.0,0
334279,4.0,3.0,27.0,0


In [7]:
# Check IDs overlap

In [8]:
len(app_train)

307511

In [9]:
print(len([element for element in app_train.index if element not in bureau.index]))
print(len([element for element in app_train.index if element not in pa.index]))
print(len([element for element in bureau.index if element not in pa.index]))

44020
16454
13984


In [10]:
# join TARGET

In [11]:
# bureau = pd.merge(bureau, app_train[["TARGET"]], left_index=True, right_index=True)
# pa = pd.merge(pa, app_train[["TARGET"]], left_index=True, right_index=True)

In [12]:
# join all ACHTUNG das ist die Schnittmenge! Zuvor sollten fehlende Daten aufbereitet werden.

In [13]:
app_train = pd.merge(app_train, bureau, left_index=True, right_index=True)
app_train = pd.merge(app_train, pa, left_index=True, right_index=True)
app_train = pd.merge(app_train, ip, left_index=True, right_index=True)
app_train = pd.merge(app_train, pos, left_index=True, right_index=True)

In [14]:
app_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,DAYS_CREDIT_0.0,DAYS_CREDIT_0.25,DAYS_CREDIT_0.5,DAYS_CREDIT_0.75,DAYS_CREDIT_1.0,CREDIT_DAY_OVERDUE_0.0,CREDIT_DAY_OVERDUE_0.25,CREDIT_DAY_OVERDUE_0.5,CREDIT_DAY_OVERDUE_0.75,CREDIT_DAY_OVERDUE_1.0,DAYS_CREDIT_ENDDATE_0.0,DAYS_CREDIT_ENDDATE_0.25,DAYS_CREDIT_ENDDATE_0.5,DAYS_CREDIT_ENDDATE_0.75,DAYS_CREDIT_ENDDATE_1.0,AMT_CREDIT_MAX_OVERDUE_0.0,AMT_CREDIT_MAX_OVERDUE_0.25,AMT_CREDIT_MAX_OVERDUE_0.5,AMT_CREDIT_MAX_OVERDUE_0.75,AMT_CREDIT_MAX_OVERDUE_1.0,CNT_CREDIT_PROLONG_0.0,CNT_CREDIT_PROLONG_0.25,CNT_CREDIT_PROLONG_0.5,CNT_CREDIT_PROLONG_0.75,CNT_CREDIT_PROLONG_1.0,AMT_CREDIT_SUM_0.0,AMT_CREDIT_SUM_0.25,AMT_CREDIT_SUM_0.5,AMT_CREDIT_SUM_0.75,AMT_CREDIT_SUM_1.0,AMT_CREDIT_SUM_DEBT_0.0,AMT_CREDIT_SUM_DEBT_0.25,AMT_CREDIT_SUM_DEBT_0.5,AMT_CREDIT_SUM_DEBT_0.75,AMT_CREDIT_SUM_DEBT_1.0,AMT_CREDIT_SUM_LIMIT_0.0,AMT_CREDIT_SUM_LIMIT_0.25,AMT_CREDIT_SUM_LIMIT_0.5,AMT_CREDIT_SUM_LIMIT_0.75,AMT_CREDIT_SUM_LIMIT_1.0,AMT_CREDIT_SUM_OVERDUE_0.0,AMT_CREDIT_SUM_OVERDUE_0.25,AMT_CREDIT_SUM_OVERDUE_0.5,AMT_CREDIT_SUM_OVERDUE_0.75,AMT_CREDIT_SUM_OVERDUE_1.0,AMT_DOWN_PAYMENT_0.0,AMT_DOWN_PAYMENT_0.25,AMT_DOWN_PAYMENT_0.5,AMT_DOWN_PAYMENT_0.75,AMT_DOWN_PAYMENT_1.0,RATE_DOWN_PAYMENT_0.0,RATE_DOWN_PAYMENT_0.25,RATE_DOWN_PAYMENT_0.5,RATE_DOWN_PAYMENT_0.75,RATE_DOWN_PAYMENT_1.0,DAYS_DECISION_0.0,DAYS_DECISION_0.25,DAYS_DECISION_0.5,DAYS_DECISION_0.75,DAYS_DECISION_1.0,SELLERPLACE_AREA_0.0,SELLERPLACE_AREA_0.25,SELLERPLACE_AREA_0.5,SELLERPLACE_AREA_0.75,SELLERPLACE_AREA_1.0,CNT_PAYMENT_0.0,CNT_PAYMENT_0.25,CNT_PAYMENT_0.5,CNT_PAYMENT_0.75,CNT_PAYMENT_1.0,DAYS_FIRST_DRAWING_0.0,DAYS_FIRST_DRAWING_0.25,DAYS_FIRST_DRAWING_0.5,DAYS_FIRST_DRAWING_0.75,DAYS_FIRST_DRAWING_1.0,DAYS_FIRST_DUE_0.0,DAYS_FIRST_DUE_0.25,DAYS_FIRST_DUE_0.5,DAYS_FIRST_DUE_0.75,DAYS_FIRST_DUE_1.0,DAYS_LAST_DUE_1ST_VERSION_0.0,DAYS_LAST_DUE_1ST_VERSION_0.25,DAYS_LAST_DUE_1ST_VERSION_0.5,DAYS_LAST_DUE_1ST_VERSION_0.75,DAYS_LAST_DUE_1ST_VERSION_1.0,AMT_APPLICATION_0.0,AMT_APPLICATION_0.25,AMT_APPLICATION_0.5,AMT_APPLICATION_0.75,AMT_APPLICATION_1.0,DAYS_TERMINATION_0.0,DAYS_TERMINATION_0.25,DAYS_TERMINATION_0.5,DAYS_TERMINATION_0.75,DAYS_TERMINATION_1.0,MIN_TIMEDIFF,MIN_AMTDIFF,MEAN_TIMEDIFF,MEAN_AMTDIFF,MAX_TIMEDIFF,MAX_AMTDIFF,CNT_CREDITS_PAST,CNT_CREDITS_CURR,SUM_PAYCOUNTS_MONTHS,SUM_DPD
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1,0,202500.0,406597.5,0.018801,-9461,-637,-3648.0,-2120,0.083037,0.262949,0.139376,0.972200,0.014900,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-1437.0,-1122.0,-1042.5,-602.75,-103.0,0.0,0.0,0.0,0.0,0.0,-1072.0,-1006.25,-424.5,79.25,780.0,0.0,0.0,40.5,3321.0,5043.645,0.0,0.0,0.0,0.0,0.0,0.0,28759.17375,54130.50,124301.25,450000.0,0.0,0.0,0.0,0.0,245781.0,0.0,0.0,0.0,7997.14125,31988.565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-606.0,-606.0,-606.0,-606.0,-606.0,500.0,500.00,500.0,500.00,500.0,24.0,24.0,24.0,24.000000,24.0,365243.0,365243.0,365243.0,365243.0,365243.0,-565.0,-565.0,-565.0,-565.0,-565.0,125.0,125.0,125.0,125.0,125.0,179055.0,179055.00,179055.0,179055.0,179055.0,-17.0,-17.0,-17.0,-17.00,-17.0,12.0,0.000,20.421053,0.000000,31.0,0.0,0.0,1.0,6.0,0
100003,0,0,270000.0,1293502.5,0.003541,-16765,-1188,-1186.0,-291,0.311267,0.622246,0.510853,0.985100,0.071400,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-2586.0,-1873.5,-1205.5,-732.75,-606.0,0.0,0.0,0.0,0.0,0.0,-2434.0,-1013.50,-480.0,-11.00,1216.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,22248.0,60051.37500,92576.25,286875.00,810000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,202500.00000,810000.000,0.0,0

In [15]:
# Ermittlung Reihenfolge

In [16]:
y = app_train["TARGET"]
x = app_train.drop(["TARGET"], axis=1)

In [17]:
# unterteilt den trainingsdatensatz in trainings- und validierungsdatensätze
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.5, random_state=0)

In [18]:
# logistisches Regressionsmodell
model = RandomForestClassifier(**TREE_PARAMS)
model.fit(x_train, y_train)

RandomForestClassifier(max_depth=20, random_state=0)

In [19]:
auc = roc_auc_score(y_test, model.predict_proba(x_test)[:,1])
fpr, tpr, threshold = roc_curve(y_test, model.predict_proba(x_test)[:,1])
print(auc)

0.7256384118448649


In [20]:
# Koeffizienten der einzelnen Klassen
coef_dict = {}
for coef, feat in zip(model.feature_importances_, x.columns.values):
    coef_dict[feat] = coef

In [21]:
# Feature Importance
d = dict(sorted(coef_dict.items(), key=lambda item: item[1], reverse=True))
order = list(d.keys())

In [22]:
#Random Forest Parameterbestimmung

In [23]:
data = {
        "auc":[],
        "auc_adj":[],
        "p" : [],
        "n": []
       }

auc_temp = 0.5
heads_gain = []
drop = []

HEADS = order
df = app_train

for index, h in enumerate(HEADS):
    
    heads_gain.append(h)
    
    X = df[heads_gain + ["TARGET"]]
    y = X["TARGET"]
    x = X.drop(["TARGET"], axis=1)
    
    model = RandomForestClassifier(**TREE_PARAMS).fit(x, y)
    
    n = len(X)
    p = len(X.columns)
    
    aucs = []

    kfold = KFold(2, shuffle=True, random_state=1)

    # enumerate splits
    for i, (train, test) in enumerate(kfold.split(x)):

        model.fit(x.iloc[train], y.iloc[train])
        auc = roc_auc_score(y.iloc[test], model.predict_proba(x.iloc[test])[:,1])
        aucs.append(auc)
    
    auc = np.mean(aucs)
    auc_adj = 1-(1-auc)*(n-1)/(n-p-1)
    
    if auc_adj > auc_temp:
        
        data["auc"].append(auc)
        data["auc_adj"].append(auc_adj)
        data["p"].append(p)
        data["n"].append(n)
        
        auc_temp = auc_adj
        
    else:
        heads_gain.remove(h)
        drop.append(h)
    
    clear_output(wait=True)
    print(auc_adj, index+1, len(HEADS), drop)

0.6596538669546141 1 125 []


KeyboardInterrupt: 

In [ ]:
result = pd.DataFrame(data)

print(result[result["auc_adj"] == result["auc_adj"].max()])
result["auc_adj"].plot()

heads_gain

In [ ]:
model = RandomForestClassifier(**TREE_PARAMS)

aucs = []

kfold = KFold(5, shuffle=True, random_state=1)
data = df[heads_gain]

# enumerate splits
for i, (train, test) in enumerate(kfold.split(data)):
    
    model.fit(x.iloc[train], y.iloc[train])
    auc = roc_auc_score(y.iloc[test], model.predict_proba(x.iloc[test])[:,1])
    aucs.append(auc)
    
    print('train: %s, test: %s, auc: %s' % (train, test, auc))

print("\n")
print("Durchschnitt: %.2f" % (np.mean(aucs)))
print("Standardabw.: %.2f" % (np.std(aucs)))
print("Varianz:      %.2f" % (np.var(aucs)))